# Convertendo uma arquivo json multinivel em um dataset estruturado.  

Nesse caso de uso fazemos uma séries de transformações para converter um JSON multinivel em um dataset simples.
Obs.: Originalmente o dado era um XML que foi convertido para JSON.

In [ ]:
import pytz, itertools, unicodedata, json, calendar
from datetime import datetime, timezone, timedelta, date
#from dateutil import relativedelta
from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *
from functools import reduce
import json

In [ ]:
# carrega os dados já filtrando pela ultima data de ingestão
last_exec_date = date.today() # Só um exemplo, Usar para isso um sistema de controle de mudança de data
folder = "path/to/data"
df = spark.read.json(folder).filter(col("DateIngest") >= last_exec_date)

In [ ]:
# Esse é um exemplo de como obter os dados do JSON de modo a gerar um DATASET na saida estruturado a partir da estruturação multinivel do JSON
# para isso usar a função explode que retorna uma linha para cada item de um array ou map e select 

# nesse exemplo também usamos o nome do arquivo que possui um time stamp em seu nome:
# pld_horario_2021-04-28 115717__2021-04-26_2021-05-02_Pg1.json

df = df.withColumn("pathfilename", input_file_name())\
      .withColumn("splitedpathfilename", split(col("pathfilename"), "/"))\
      .withColumn("filename", element_at(col('splitedpathfilename'), -1))\
      .withColumn('plds',explode(col('soapenv:Envelope.soapenv:Body.bm:listarPLDResponse.bm:plds.bm:pld')))\
      .select(["*", "plds.bo:vigencia.*", "plds.bo:valores.bo:valor"])\
      .withColumn('valores',explode(col('bo:valor')))\
      .drop("bo:valor")\
      .select(["*", "valores.bo:indicadorRedeEletrica", "valores.bo:tipo", "valores.bo:submercado.*" ])\
      .withColumnRenamed("bo:codigo", "sg_regiao")\
      .select(["*", "valores.bo:valor.bo:codigo", "valores.bo:valor.bo:valor"])\
      .drop("soapenv:Envelope").drop("plds").drop("valores").drop("pathfilename").drop("splitedpathfilename").drop("bo:tipo")\
      .drop("yearingest").drop("monthingest").drop("dateingest")\
      .withColumn("ts_ingest_utc", regexp_replace(split(split(col("filename"), "__").getItem(0), "_").getItem(2), "%20", " "))\
      .drop("filename")\
      .withColumn("ts_inicio_vigencia_utc", to_timestamp("bo:inicio"))\
      .withColumn("ts_inicio_vigencia_utc", to_timestamp("bo:inicio"))\
      .withColumn("ts_fim_vigencia_utc", to_timestamp("bo:fim"))\
      .withColumn("sg_timezone", substring(col("bo:fim"), 20,6))\
      .withColumn("ts_ingest_utc", to_timestamp("ts_ingest_utc", "yyyy-MM-dd HHmmss"))\
      .withColumn('dt_inicio_vigencia_local', to_date(from_utc_timestamp("ts_inicio_vigencia_utc", col("sg_timezone"))))\
      .withColumn('hr_inicio_vigencia_local', date_format(from_utc_timestamp("ts_inicio_vigencia_utc", col("sg_timezone")), "HH:mm:ss"))\
      .withColumn('dt_fim_vigencia_local', to_date(from_utc_timestamp("ts_fim_vigencia_utc", col("sg_timezone"))))\
      .withColumn('hr_fim_vigencia_local', date_format(from_utc_timestamp("ts_fim_vigencia_utc", col("sg_timezone")), "HH:mm:ss"))\
      .drop("bo:inicio").drop("bo:fim")\
      .withColumnRenamed("bo:codigo", "sg_moeda")\
      .withColumnRenamed("bo:nome", "nm_regiao")\
      .withColumnRenamed("bo:valor", "vl_preco_kwh")\
      .withColumn("fl_indicador_rede_eletrica", col("bo:indicadorRedeEletrica").cast('boolean'))\
      .select(["ts_inicio_vigencia_utc", "sg_regiao", "ts_fim_vigencia_utc", "dt_inicio_vigencia_local", "hr_inicio_vigencia_local",\
               "dt_fim_vigencia_local", "hr_fim_vigencia_local", "fl_indicador_rede_eletrica", "nm_regiao", "sg_moeda", \
               "vl_preco_kwh", "sg_timezone", "ts_ingest_utc"])

Exemplo de arquivo JSON na entrada

```json
{"soapenv:Envelope":{"@xmlns:hdr":"http://xmlns.energia.org.br/MH/v1","@xmlns:soapenv":"http://schemas.xmlsoap.org/soap/envelope/","@xmlns:xsi":"http://www.w3.org/2001/XMLSchema-instance","soapenv:Header":{"mh:messageHeader":{"@xmlns:mh":"http://xmlns.energia.org.br/MH/v1","mh:codigoPerfilAgente":"3801","mh:transactionId":"2c00f25e-2986-4f5d-991e-a6550a22f3a5"},"hdr:paginacao":{"hdr:numero":"1","hdr:quantidadeItens":"50","hdr:totalPaginas":"1","hdr:quantidadeTotalItens":"24"}},"soapenv:Body":{"bm:listarPLDResponse":{"@xmlns:bo":"http://xmlns.energia.org.br/BO/v1","@xmlns:bm":"http://xmlns.energia.org.br/BM/v1","bm:plds":{"bm:pld":[{"bo:vigencia":{"bo:inicio":"2021-10-08T00:00:00-03:00","bo:fim":"2021-10-08T01:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"428.65"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"428.65"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"428.65"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"428.65"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T01:00:00-03:00","bo:fim":"2021-10-08T02:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"420.05"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"420.05"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"420.05"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"420.05"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T02:00:00-03:00","bo:fim":"2021-10-08T03:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"416.22"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"416.22"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"416.22"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"416.22"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T03:00:00-03:00","bo:fim":"2021-10-08T04:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"417.65"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"417.65"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"417.65"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"417.65"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T04:00:00-03:00","bo:fim":"2021-10-08T05:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"417.25"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"417.25"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"417.25"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"417.25"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T05:00:00-03:00","bo:fim":"2021-10-08T06:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"418.73"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"418.73"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"418.73"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"418.73"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T06:00:00-03:00","bo:fim":"2021-10-08T07:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"425.97"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"425.97"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"425.97"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"425.97"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T07:00:00-03:00","bo:fim":"2021-10-08T08:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"432.88"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"432.88"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"432.88"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"432.88"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T08:00:00-03:00","bo:fim":"2021-10-08T09:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.56"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.56"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.56"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.56"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T09:00:00-03:00","bo:fim":"2021-10-08T10:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"436.66"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"436.66"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"436.66"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"436.66"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T10:00:00-03:00","bo:fim":"2021-10-08T11:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.65"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.65"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.65"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.65"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T11:00:00-03:00","bo:fim":"2021-10-08T12:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.11"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.11"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.11"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.11"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T12:00:00-03:00","bo:fim":"2021-10-08T13:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"434.36"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"434.36"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"434.36"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"434.36"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T13:00:00-03:00","bo:fim":"2021-10-08T14:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.79"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.79"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.79"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.79"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T14:00:00-03:00","bo:fim":"2021-10-08T15:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"441.09"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"441.09"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"441.09"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"441.09"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T15:00:00-03:00","bo:fim":"2021-10-08T16:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"443.93"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"443.93"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"443.93"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"443.93"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T16:00:00-03:00","bo:fim":"2021-10-08T17:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"444.72"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"444.72"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"444.72"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"444.72"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T17:00:00-03:00","bo:fim":"2021-10-08T18:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"438.13"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"438.13"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"438.13"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"438.13"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T18:00:00-03:00","bo:fim":"2021-10-08T19:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"445.47"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"445.47"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"445.47"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"445.47"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T19:00:00-03:00","bo:fim":"2021-10-08T20:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"443.01"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"443.01"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"443.01"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"443.01"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T20:00:00-03:00","bo:fim":"2021-10-08T21:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"438.84"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"438.84"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"438.84"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"438.84"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T21:00:00-03:00","bo:fim":"2021-10-08T22:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"439.81"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"439.81"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"439.81"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"439.81"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T22:00:00-03:00","bo:fim":"2021-10-08T23:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.16"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.16"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.16"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"435.16"}}]}},{"bo:vigencia":{"bo:inicio":"2021-10-08T23:00:00-03:00","bo:fim":"2021-10-09T00:00:00-03:00"},"bo:valores":{"bo:valor":[{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"1","bo:nome":"SUDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"428.7"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"2","bo:nome":"SUL"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"428.7"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"3","bo:nome":"NORDESTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"428.7"}},{"bo:indicadorRedeEletrica":"false","bo:submercado":{"bo:codigo":"4","bo:nome":"NORTE"},"bo:tipo":"HORARIO","bo:valor":{"bo:codigo":"BRL","bo:valor":"428.7"}}]}}]}}}}}
```